
## Work in progress - openCoesione with Comune as key 

In [1]:
## Runs with "daf" enviroment in repo

%config Completer.use_jedi = False #fix TAB slowness with big frames

import numpy as np
import pandas as pd
import seaborn as sns
import glob
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)


def load_and_stack_data(prefix):
    fileList = glob.glob('./../pac_opencoesione/%s*.csv' % prefix)
    
    if prefix == 'progetti':
        dtypesDict = pd.read_csv(
            './openCoesione_dtypes.csv', index_col='Variabile', squeeze=True).to_dict()
        dateCols = [name for name in dtypesDict.keys() if dtypesDict[name] == 'datetime64']
        for name in dateCols: del dtypesDict[name]
        frames = list(map(lambda f: pd.read_csv(
            f, sep=';', dtype=dtypesDict, index_col=0, na_values=[' ', '  ']), fileList))
        data = pd.concat(frames, axis=0)
        data[dateCols] = data[dateCols].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
    else:
        # infer types for other datasets (temporary)
        frames = list(map(lambda f: pd.read_csv(f, sep=';', index_col=0), fileList))
        data = pd.concat(frames, axis=0)
        
    return data

In [2]:
# load datasets
progetti = load_and_stack_data('progetti')
pagamenti = load_and_stack_data('pagamenti')
luoghi = load_and_stack_data('localizzazioni')
soggetti = load_and_stack_data('soggetti')


/home/davide/anaconda3/envs/daf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (12,22,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/davide/anaconda3/envs/daf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (12,22,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/davide/anaconda3/envs/daf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/davide/anaconda3/envs/daf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (4,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(cod

In [3]:
# join: pagamenti + (progetti + soggetti==2) e verifica luoghi

newUnion = progetti.join(soggetti[soggetti.SOGG_COD_RUOLO == 2])




In [4]:
newUnion.head()

,COD_ATECO,COD_AZIONE,COD_GRANDE_PROGETTO,COD_LINEA,COD_PROCED_ATTIVAZIONE,COD_STRUMENTO,COD_TIPO_PROCED_ATTIVAZIONE,COSTO_REALIZZATO,COSTO_RENDICONTABILE_UE,CUP,...,DESCR_FORMA_GIURIDICA_SOGG,COD_COMUNE_SEDE_SOGG,INDIRIZZO_SOGG,CAP_SOGG,COD_ATECO_SOGG,DESCRIZIONE_ATECO_SOGG,COD_ADDETTI_SOGG,DESCR_ADDETTI_SOGG,COD_DIMENSIONE_SOGG,DESCR_DIMENSIONE_SOGG
COD_LOCALE_PROGETTO,,,,,,,,,,,,,,,,,,,,,
17EM0223544,G-46.51.00,NaN,NaN,NaN,PATT6843,NaN,1,NaN,44340.0,E68I08000480005,...,Regione,,,,O-84.11.10,AttivitÃÂ degli organi legislativi ed esecut...,,,NaN,
17EM0228885,J-62.01.00,NaN,NaN,NaN,PATT6843,NaN,1,NaN,91148.0,E38I08000140005,...,Regione,,,,O-84.11.10,AttivitÃÂ degli organi legislativi ed esecut...,,,NaN,
17EM0235733,C-27.90.09,NaN,NaN,NaN,PATT6843,NaN,1,NaN,36000.0,E88I08000020005,...,Regione,,,,O-84.11.10,AttivitÃÂ degli organi legislativi ed esecut...,,,NaN,
17EM0235756,C-28.23.09,NaN,NaN,NaN,PATT6843,NaN,1,NaN,66000.0,E38I08000610005,...,Regione,,,,O-84.11.10,AttivitÃÂ degli organi legislativi ed esecut...,,,NaN,
17EM0237525,C-32.40.10,NaN,NaN,NaN,PATT6843,NaN,1,NaN,57200.0,E68I08000630005,...,Regione,,,,O-84.11.10,AttivitÃÂ degli organi legislativi ed esecut...,,,NaN,


In [5]:
# Summary on beneficiary types
soggetti.groupby('DESCR_FORMA_GIURIDICA_SOGG')['OC_CODICE_FISCALE_SOGG'].nunique().sort_values(ascending=False)

DESCR_FORMA_GIURIDICA_SOGG
Società  per azioni                                                                                                        22420
Imprenditore individuale non agricolo                                                                                       6239
Società  in accomandita semplice                                                                                            5588
Società  in nome collettivo                                                                                                 5443
Comune                                                                                                                      5408
Società  di fatto o irregolare, comunione ereditaria                                                                        4138
Istituto e scuola pubblica di ogni ordine e grado                                                                           3126
Lavoratore autonomo                                                   